# Exploring AWS Public Blockchain Dataset

In [15]:
import pandas as pd

# Path to your local Parquet file
file_path = "./v1.0/btc/blocks/date=2021-01-28/part-00000-07e13b0c-d837-45e3-9d7f-1352c478ed55-c000.snappy.parquet"

# Read the Parquet file
df = pd.read_parquet(file_path)

# Display the first few rows
print(df.head())

                                                hash     version  \
0  00000000000000000003172093fcff9705da7519048ebe...   536870912   
1  00000000000000000004b3835952e30039dcbccb1b0ba5...   805298176   
2  00000000000000000000109cc8faade73eaded7d9ebf29...  1073733632   
3  000000000000000000049dae6ac421012c91d44b1ea7af...   671080448   
4  000000000000000000056cce6e8dba8eb5ff9750f1358b...   541065216   

           mediantime       nonce      bits    difficulty  \
0 2021-01-28 17:16:02  2786007068  170d8457  2.082353e+13   
1 2021-01-28 03:48:43  2759903233  170d8457  2.082353e+13   
2 2021-01-28 08:23:36  2323570707  170d8457  2.082353e+13   
3 2021-01-28 14:39:32  2904890703  170d8457  2.082353e+13   
4 2021-01-28 17:53:17  3846823593  170d8457  2.082353e+13   

                                           chainwork  \
0  000000000000000000000000000000000000000018b0eb...   
1  000000000000000000000000000000000000000018aa2f...   
2  000000000000000000000000000000000000000018ac7a...   


In [11]:
print(df.info())

# Summary statistics
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   hash               145 non-null    object        
 1   version            145 non-null    int64         
 2   mediantime         145 non-null    datetime64[ns]
 3   nonce              145 non-null    int64         
 4   bits               145 non-null    object        
 5   difficulty         145 non-null    float64       
 6   chainwork          145 non-null    object        
 7   previousblockhash  145 non-null    object        
 8   size               145 non-null    int64         
 9   weight             145 non-null    int64         
 10  coinbase_param     145 non-null    object        
 11  number             145 non-null    int64         
 12  transaction_count  145 non-null    int64         
 13  merkle_root        145 non-null    object        
 14  stripped_s

In [19]:
# Sort the DataFrame by the 'timestamp' column
df_sorted = df.sort_values(by='timestamp')

# Reset the index after sorting
df_sorted.reset_index(drop=True, inplace=True)

# Verify the order
print(df_sorted['timestamp'])


0     2021-01-28 00:34:47
1     2021-01-28 00:49:37
2     2021-01-28 01:01:26
3     2021-01-28 01:03:35
4     2021-01-28 01:32:45
              ...        
140   2021-01-28 23:27:32
141   2021-01-28 23:29:18
142   2021-01-28 23:30:21
143   2021-01-28 23:40:20
144   2021-01-28 23:41:13
Name: timestamp, Length: 145, dtype: datetime64[ns]


In [23]:
df_sorted.iloc[0,0]

'000000000000000000033603ee318f49b02dd2bc1c9e400e6e20d81fe184f338'

In [24]:
df_sorted.iloc[0,:]

hash                 000000000000000000033603ee318f49b02dd2bc1c9e40...
version                                                      939515904
mediantime                                         2021-01-27 23:26:59
nonce                                                        337490073
bits                                                          170d8457
difficulty                                       20823531150111.519531
chainwork            000000000000000000000000000000000000000018a8a1...
previousblockhash    00000000000000000002304bc1c3e870d0bdb3c5e315b8...
size                                                           1386156
weight                                                         3998637
coinbase_param       0343310a2cfabe6d6d065aeed33ce7ccc3cc4ac2860ca8...
number                                                          667971
transaction_count                                                 2975
merkle_root          5f68f9fdfbb1fcffd845c0c8dd120fc65c13cdb0a04c7a...
stripp

# Exploring Blockchair Data Dump

In [22]:
from datetime import date, timedelta
import os
import requests
import pandas as pd
import gzip
import shutil
import csv

def download_and_combine_files(types, start_date, end_date, output_filename="combined_data"):
    """Downloads files and combines them into a single gzipped TSV file."""

    for type in types:
        final = []
        current_date = start_date
        while current_date <= end_date:
            formatted_date = current_date.strftime("%Y%m%d")
            file_url = f"https://gz.blockchair.com/bitcoin/{type}/blockchair_bitcoin_{type}_{formatted_date}.tsv.gz"

            response = requests.get(file_url, stream=True)
            

            if response.status_code == 200:
                temp_filename = f"temp_{formatted_date}.tsv.gz" # Temporary files
                with open(temp_filename, "wb") as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                with gzip.open(temp_filename, "rt") as f: 
                    l = list(csv.reader(f, delimiter="\t"))
                    final += (l[1:] if final else l)  
                os.remove(temp_filename)   
                print(f"Downloaded: {file_url}")
            else:
                print(f"Error downloading {file_url}: {response.status_code}")

            current_date += timedelta(days=1)
        
        df = pd.DataFrame(final[1:], columns=final[0])
        df.to_csv(f"{output_filename}_{type}.csv", index=False)
    

def read_combined_data(filename="combined_data_inputs.csv"):
    """Reads combined data from a gzipped TSV file into a Pandas DataFrame."""
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except pd.errors.EmptyDataError:
        print(f"Error: File '{filename}' is empty.")
        return None
    except Exception as e: # Catch other potential exceptions during reading
        print(f"An error occurred while reading the file: {e}")
        return None

# usage
types = ["inputs", "transactions", "outputs"]
start_date = date(2010, 2, 1)
end_date = date(2010, 2, 10)

download_and_combine_files(types, start_date, end_date)

Downloaded: https://gz.blockchair.com/bitcoin/inputs/blockchair_bitcoin_inputs_20100201.tsv.gz
Downloaded: https://gz.blockchair.com/bitcoin/inputs/blockchair_bitcoin_inputs_20100202.tsv.gz
Downloaded: https://gz.blockchair.com/bitcoin/inputs/blockchair_bitcoin_inputs_20100203.tsv.gz
Downloaded: https://gz.blockchair.com/bitcoin/inputs/blockchair_bitcoin_inputs_20100204.tsv.gz
Downloaded: https://gz.blockchair.com/bitcoin/inputs/blockchair_bitcoin_inputs_20100205.tsv.gz
Downloaded: https://gz.blockchair.com/bitcoin/inputs/blockchair_bitcoin_inputs_20100206.tsv.gz
Downloaded: https://gz.blockchair.com/bitcoin/inputs/blockchair_bitcoin_inputs_20100207.tsv.gz
Downloaded: https://gz.blockchair.com/bitcoin/inputs/blockchair_bitcoin_inputs_20100208.tsv.gz
Downloaded: https://gz.blockchair.com/bitcoin/inputs/blockchair_bitcoin_inputs_20100209.tsv.gz
Downloaded: https://gz.blockchair.com/bitcoin/inputs/blockchair_bitcoin_inputs_20100210.tsv.gz
Downloaded: https://gz.blockchair.com/bitcoin/tran

In [28]:
print(read_combined_data("combined_data_inputs.csv").shape)
read_combined_data("combined_data_inputs.csv").head()

(1522, 21)


,block_id,transaction_hash,index,time,value,value_usd,recipient,type,script_hex,is_from_coinbase,...,spending_block_id,spending_transaction_hash,spending_index,spending_time,spending_value_usd,spending_sequence,spending_signature_hex,spending_witness,lifespan,cdd
0,37389,d9a4176d14dbb00d6b798a30b32243af455be181f3cd2b...,0,2010-01-31 08:28:21,35000000000,3.50,1JvrS4xpuKiiiB51w1Mwewrspx2BVBkJSr,pubkey,4104e3a564a42717a9d4c15f47b33adf96d2a6286bf6dd...,0,...,37503,a41bb1b3f8ce6b1d9cf7bd7a8707efeb7316dd5ba11549...,0,2010-02-01 00:41:31,3.50,4294967295,493046022100c602ba8ef022965fc25dc3a33fe1a6e5ff...,NaN,58390,236.533565
1,37503,a41bb1b3f8ce6b1d9cf7bd7a8707efeb7316dd5ba11549...,1,2010-02-01 00:41:31,34900000000,3.49,14DBT1FZkzR7SjZv4cghhTJxJYCwDP3M24,pubkey,4104dcf9e313efd7aec54c423d25a559a83311ec3574f5...,0,...,37513,8697331c3124c8a4cf2f43afb5732374ea13769e42f10a...,0,2010-02-01 01:43:32,3.49,4294967295,483045022100eaa5542714d1e31eada58c31e6ac77774a...,NaN,3721,15.030428
2,37503,a41bb1b3f8ce6b1d9cf7bd7a8707efeb7316dd5ba11549...,0,2010-02-01 00:41:31,100000000,0.01,14dZLztXXGSYuZDHBqHe7iXiCGNgLTfsdY,pubkeyhash,76a91427d25a1ff9a6da31eeb991c48bb6cd95191a6b2c...,0,...,37514,60748843bb13a8239a91505cbb4eeabc8b3d1f6ed1fdf9...,0,2010-02-01 01:47:30,0.01,4294967295,4830450221009b9e20ad2cf6715af44cadf588f14155cc...,NaN,3959,0.045822
3,37513,8697331c3124c8a4cf2f43afb5732374ea13769e42f10a...,1,2010-02-01 01:43:32,20000000000,2.00,16KkRwwdyoZPYpcBqUbM8YrifGyLjcJdj7,pubkey,4104d4ae60e43fd37ed680f40f86576f54fc57f61d3da0...,0,...,37514,acb2e895a9b3664cfcfb13dd0321ef0c1205cf2aa08268...,0,2010-02-01 01:47:30,2.00,4294967295,4730440220106c316c22d4bf8c970f8e4b1af1edd95121...,NaN,238,0.550926
4,37513,8697331c3124c8a4cf2f43afb5732374ea13769e42f10a...,0,2010-02-01 01:43:32,14900000000,1.49,1C3XZVD7J3UmNCFZYHiTRGL2X726ACWmW4,pubkey,41042f40514a99be39b3e94345455f78c4dfbbcd7f9cc6...,0,...,37521,a55aab1f308278fb57a87617645f3f9a7bf9cd97b0aab3...,0,2010-02-01 02:40:35,1.49,4294967295,473044022044547a1a1471a1cfcf3c438b34e46e7b5b67...,NaN,3423,5.903090


In [30]:
print(read_combined_data("combined_data_outputs.csv").shape)
read_combined_data("combined_data_outputs.csv").head()

(2102, 11)


,block_id,transaction_hash,index,time,value,value_usd,recipient,type,script_hex,is_from_coinbase,is_spendable
0,37494,faf705b1dd0c22dc55b77fc436618118800ea9bd22f892...,0,2010-02-01 00:08:21,5000000000,0.5,17PeCY98XRyEK9DPk5c1PQoMj8muwJs1Q6,pubkey,4104f95172805de937ed767b5df88a5b1042e1439b3067...,1,1
1,37495,825b31b5c08174c85530c5a643cbcce2781d4b4ae6a9ee...,0,2010-02-01 00:10:53,5000000000,0.5,18Tt2YoxAwJBtphjfiGm8Zr3K8dL7edEWh,pubkey,410498b7e6384de6ec2f71391cd6c9338683aec655e4fe...,1,1
2,37496,f480fd66255fe9bafd9b730effeace78182c0eb5c06e95...,0,2010-02-01 00:12:59,5000000000,0.5,1M1L1qBstM4fEPLCLQDHpBtZANHosAGo1j,pubkey,41047db6122b297f34e5c4264adcd05223f6eb097f9905...,1,1
3,37497,d9d0a615902ad038de11f7fe8e49d044d8d1a77791e663...,0,2010-02-01 00:15:31,5000000000,0.5,13vzopiP2ZAxiM7EhZzHjrroQh2dmgFvJF,pubkey,41043d54556a5e9f8905b186618ac0e527c4f712231bd7...,1,1
4,37498,ff5c63a108eeea781b25e8ed90ac98204b528d29480f6e...,0,2010-02-01 00:17:36,5000000000,0.5,1C7C4GidhcfkjfpQBrbb12JShiQy4MtaxY,pubkey,410440df56a3fb9272ce8839d7e8ebdf8edf830ad8b7d8...,1,1


In [29]:
print(read_combined_data("combined_data_transactions.csv").shape)
read_combined_data("combined_data_transactions.csv").head()

(2091, 22)


,block_id,hash,time,size,weight,version,lock_time,is_coinbase,has_witness,input_count,...,input_total_usd,output_total,output_total_usd,fee,fee_usd,fee_per_kb,fee_per_kb_usd,fee_per_kwu,fee_per_kwu_usd,cdd_total
0,37494,faf705b1dd0c22dc55b77fc436618118800ea9bd22f892...,2010-02-01 00:08:21,135,540,1,0,1,0,1,...,0.0,5000000000,0.5,0,0.0,0,0.0,0,0,0.0
1,37495,825b31b5c08174c85530c5a643cbcce2781d4b4ae6a9ee...,2010-02-01 00:10:53,135,540,1,0,1,0,1,...,0.0,5000000000,0.5,0,0.0,0,0.0,0,0,0.0
2,37496,f480fd66255fe9bafd9b730effeace78182c0eb5c06e95...,2010-02-01 00:12:59,134,536,1,0,1,0,1,...,0.0,5000000000,0.5,0,0.0,0,0.0,0,0,0.0
3,37497,d9d0a615902ad038de11f7fe8e49d044d8d1a77791e663...,2010-02-01 00:15:31,135,540,1,0,1,0,1,...,0.0,5000000000,0.5,0,0.0,0,0.0,0,0,0.0
4,37498,ff5c63a108eeea781b25e8ed90ac98204b528d29480f6e...,2010-02-01 00:17:36,134,536,1,0,1,0,1,...,0.0,5000000000,0.5,0,0.0,0,0.0,0,0,0.0
